# Fediporter
Jupyter notebook to migrate content like Tweets oder Mastodon posts to Mastodon.

Only works with instances that are patched to allow backdated posts through the API. More info: https://github.com/lucahammer/fediporter

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import json

In [163]:
with open('config.json', 'r') as f:
    config = json.loads(f.read())

API_BASE_URL = config['mastodon_url']
MASTODON_BEARER =  "oyufS1RbwqgIMlLcudOtA33F4N5YZrkDHiHDlw5Y-dA"  #config['mastodon_bearer']
HEADERS = headers={'Authorization': f'Bearer {MASTODON_BEARER}'}

# Test Mastodon bearer token
url = f"{API_BASE_URL}/api/v1/apps/verify_credentials"
r = requests.get(url, headers=HEADERS)
print(r.text)

{"name":"Imported from det.social/@Luca","website":"https://det.social/@luca","vapid_key":"uXL829wxgH9wYavQxkARcWv6fiNUZ94B"}


In [5]:
def post_status(data):
    HEADERS = {'Authorization': f'Bearer {MASTODON_BEARER}'}
    url = f"{API_BASE_URL}/api/v1/statuses"
    r = requests.post(url, 
                      data=data, 
                      headers=HEADERS)
    return r.json()

def load_mastodon_posts(file):
    with open(file, 'r') as f:
        posts = json.loads(f.read())['orderedItems']
    return(posts)

def fix_text(text, tags):
    text = text.replace('</p><p>', '++breakme++</p><p>') #add placeholder to replace with linebreaks to keep paragraphs (it's weird)
    text = BeautifulSoup(text, 'html.parser').get_text()
    tags = [tag for tag in tags if tag['type'] == 'Mention']
    for tag in tags:
        name = f"@{tag['name'].split('@')[1]} "
        text = text.replace(name,f"{tag['name']} ")
    text = text.replace('++breakme++', '\n\n')
    return (text)

In [6]:
posts = load_mastodon_posts('detsocial/outbox.json')

In [258]:
masto_dict = {}

In [262]:
for post in tqdm(posts):
    if 'contentMap' in post['object'] and post['id'] not in masto_dict and (('inReplyTo' in post and '/luca/' in post['inReplyTo']) or 'inReplyTo' not in post):
        if 'en' in post['object']['contentMap']:
            toot = {
                'status': fix_text(post['object']['contentMap']['en'], post['object']['tag']),
                'language': 'en'}
        if 'de' in post['object']['contentMap']:
            toot = {
                'status': fix_text(post['object']['contentMap']['de'], post['object']['tag']),
                'language': 'de'}
        toot['created_at'] = post['object']['published']
        if post['to'][0].endswith('/followers') and len(post['cc']) > 0 and post['cc'][0].endswith('#Public'):
            toot['visibility'] = 'unlisted'
        elif post['to'][0].endswith('#Public'):
            toot['visibility'] = 'public'
        elif post['to'][0].endswith('/followers') and len(post['cc']) > 0 and not post['cc'][0].endswith('#Public'):
            toot['visibility'] = 'private'
        else:
            toot['visibility'] = 'direct'
        
        if 'attachment' in post['object'] and len(post['object']['attachment']) > 0:
            #print(post['object']['attachment'])
            media_ids = []
            for attachment in post['object']['attachment']:           
                image_path = f"detsocial/home/det/live/public/system/{attachment['url']}"
                file = open(image_path, 'rb')
                data = file.read()
                url = f"{API_BASE_URL}/api/v2/media"
                files={'file': (image_path, data, 'application/octet-stream')}
                if 'name' in attachment:
                    values = {'description' : attachment['name']}
                    r = requests.post(url, files=files, data=values, headers=HEADERS)
                else:
                    r = requests.post(url, files=files, headers=HEADERS)
                json_data = r.json()
                media_ids.append(json_data['id'])
            toot['media_ids[]'] = media_ids
        
        if 'inReplyTo' in post['object'] and post['object']['inReplyTo'] is not None and '/luca/' in post['object']['inReplyTo']:
            # if post is part of a thread, get ID of previous post
            try:
                toot['in_reply_to_id'] = masto_dict.get(post['object']['inReplyTo'])
            except:
                print(post)

        #print(toot)        
        posted = post_status(toot)
        #print(posted)
        
        masto_dict[post['object']['id']] = posted['id']
    else:
        pass
        #print('skipped')
    

In [280]:
# Manually adding new statuses

toot = {}
media_ids = []

toot = {
    'status': '''xxx''',
    'language': 'en'}

image_path = f"xxx.png"
file = open(image_path, 'rb')
data = file.read()
url = f"{API_BASE_URL}/api/v2/media"
files={'file': (image_path, data, 'application/octet-stream')}
r = requests.post(url, files=files, data={'description' : '''xxx'''}, headers=HEADERS)
json_data = r.json()
media_ids.append(json_data['id'])
toot['media_ids[]'] = media_ids
toot['in_reply_to_id'] = 'xxx'
toot['created_at'] = "2022-12-25T20:21:00Z"

In [ ]:
post_status(toot)

# todo
- content warnings?
- URLs?

In [257]:
# delete the posted posts because something went wrong

def delete_status(status_id):
    HEADERS = {'Authorization': f'Bearer {MASTODON_BEARER}'}
    url = f"{API_BASE_URL}/api/v1/statuses/{status_id}"
    r = requests.delete(url, 
                      headers=HEADERS)
    return r.json()

for thing in tqdm(masto_dict.values()):
    x = (delete_status(thing))